# YOLOv8 vs YOLOv12 Comparison

Compare model performance metrics from training results.


In [ ]:
import json
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

# Setup paths
root = Path('..').resolve() if Path.cwd().name == 'notebooks' else Path('.')
results_dir = root / 'runs' / 'detect'

# Find latest YOLOv8 and YOLOv12 runs
yolov8_runs = sorted(results_dir.glob('yolov8_dental*'))
yolov12_runs = sorted(results_dir.glob('yolov12_dental*'))

if not yolov8_runs:
    raise FileNotFoundError("No YOLOv8 training results found. Train YOLOv8 first.")
if not yolov12_runs:
    raise FileNotFoundError("No YOLOv12 training results found. Train YOLOv12 first.")

# Get latest runs
yolov8_latest = yolov8_runs[-1]
yolov12_latest = yolov12_runs[-1]

print(f"YOLOv8 results: {yolov8_latest.name}")
print(f"YOLOv12 results: {yolov12_latest.name}")

# Read results.csv from each
v8_csv = yolov8_latest / 'results.csv'
v12_csv = yolov12_latest / 'results.csv'

# Check if CSV files exist
if not v8_csv.exists():
    raise FileNotFoundError(f"YOLOv8 results.csv not found. Training may have been interrupted. Re-run notebook 02.")
if not v12_csv.exists():
    raise FileNotFoundError(f"YOLOv12 results.csv not found. Run notebook 03 to train YOLOv12 first.")

df_v8 = pd.read_csv(v8_csv)
df_v12 = pd.read_csv(v12_csv)

# Get final epoch metrics
v8_final = df_v8.iloc[-1]
v12_final = df_v12.iloc[-1]

# Create comparison dataframe
comparison = pd.DataFrame({
    'Model': ['YOLOv8', 'YOLOv12'],
    'mAP50': [v8_final['metrics/mAP50(B)'], v12_final['metrics/mAP50(B)']],
    'mAP50-95': [v8_final['metrics/mAP50-95(B)'], v12_final['metrics/mAP50-95(B)']],
    'Precision': [v8_final['metrics/precision(B)'], v12_final['metrics/precision(B)']],
    'Recall': [v8_final['metrics/recall(B)'], v12_final['metrics/recall(B)']],
})

print("\n" + "="*60)
print("MODEL COMPARISON")
print("="*60)
print(comparison.to_string(index=False))


_IncompleteInputError: incomplete input (2597278955.py, line 20)

## Visualization: Performance Metrics

Compare mAP, Precision, and Recall between models.


In [2]:
# Bar chart comparison
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

metrics = ['mAP50', 'mAP50-95', 'Precision', 'Recall']
colors = ['tab:blue', 'tab:orange']

for idx, metric in enumerate(metrics):
    ax = axes[idx // 2, idx % 2]
    ax.bar(comparison['Model'], comparison[metric], color=colors)
    ax.set_title(f'{metric} Comparison')
    ax.set_ylabel('Score')
    ax.set_ylim(0, 1)
    ax.grid(True, alpha=0.3)
    
    # Add value labels on bars
    for i, v in enumerate(comparison[metric]):
        ax.text(i, v + 0.02, f'{v:.3f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

# Determine winner
print("\n" + "="*60)
print("WINNER ANALYSIS")
print("="*60)

for metric in metrics:
    winner_idx = comparison[metric].idxmax()
    winner = comparison.loc[winner_idx, 'Model']
    value = comparison.loc[winner_idx, metric]
    print(f"{metric:12s}: {winner} ({value:.3f})")


NameError: name 'plt' is not defined

In [ ]:
# Plot training curves
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Box loss
axes[0].plot(df_v8['train/box_loss'], label='YOLOv8', marker='o')
axes[0].plot(df_v12['train/box_loss'], label='YOLOv12', marker='s')
axes[0].set_title('Box Loss')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Class loss
axes[1].plot(df_v8['train/cls_loss'], label='YOLOv8', marker='o')
axes[1].plot(df_v12['train/cls_loss'], label='YOLOv12', marker='s')
axes[1].set_title('Classification Loss')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# mAP50
axes[2].plot(df_v8['metrics/mAP50(B)'], label='YOLOv8', marker='o')
axes[2].plot(df_v12['metrics/mAP50(B)'], label='YOLOv12', marker='s')
axes[2].set_title('mAP@0.5')
axes[2].set_xlabel('Epoch')
axes[2].set_ylabel('mAP')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# Compare model sizes
models_dir = root / 'models'
v8_model = models_dir / 'yolov8_best.pt'
v12_model = models_dir / 'yolov12_best.pt'

size_comparison = pd.DataFrame({
    'Model': ['YOLOv8', 'YOLOv12'],
    'Size (MB)': [
        v8_model.stat().st_size / (1024*1024) if v8_model.exists() else 0,
        v12_model.stat().st_size / (1024*1024) if v12_model.exists() else 0
    ]
})

print("\nModel File Sizes:")
print(size_comparison.to_string(index=False))

# Bar chart for size
plt.figure(figsize=(6, 4))
plt.bar(size_comparison['Model'], size_comparison['Size (MB)'], color=colors)
plt.title('Model File Size Comparison')
plt.ylabel('Size (MB)')
plt.grid(True, alpha=0.3)

for i, v in enumerate(size_comparison['Size (MB)']):
    plt.text(i, v + 0.1, f'{v:.1f} MB', ha='center', va='bottom')

plt.tight_layout()
plt.show()


In [ ]:
# Final recommendation
print("\n" + "="*60)
print("FINAL RECOMMENDATION")
print("="*60)

# Calculate overall score (weighted average)
weights = {'mAP50': 0.4, 'Precision': 0.3, 'Recall': 0.3}
comparison['Overall Score'] = (
    comparison['mAP50'] * weights['mAP50'] +
    comparison['Precision'] * weights['Precision'] +
    comparison['Recall'] * weights['Recall']
)

best_idx = comparison['Overall Score'].idxmax()
best_model = comparison.loc[best_idx, 'Model']
best_score = comparison.loc[best_idx, 'Overall Score']

print(f"\nBest Model: {best_model}")
print(f"Overall Score: {best_score:.3f}")
print(f"\nBased on:")
print(f"  - mAP50: {comparison.loc[best_idx, 'mAP50']:.3f}")
print(f"  - Precision: {comparison.loc[best_idx, 'Precision']:.3f}")
print(f"  - Recall: {comparison.loc[best_idx, 'Recall']:.3f}")

print(f"\nRecommendation: Use {best_model} for deployment")


## Conclusion

Summary of which model performs better for dental X-ray cavity detection.


## Model Size Comparison

Compare model file sizes and parameters.


## Training Curves Comparison

Compare loss curves over epochs.
